In [1]:
import ee
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

from geeml.utils import eeprint

In [9]:
import os
os.chdir(r"C:\Users\coach\myfiles\postdoc\Fire\code\Burn_Area_Mapping\src\components")
os.getcwd()

'C:\\Users\\coach\\myfiles\\postdoc\\Fire\\code\\Burn_Area_Mapping\\src\\components'

In [15]:
from src.logger import logging
from data_preprocessing import preProcessyCollection
from data_preprocessing import preProcessXCollection
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from exception import customException
import sys

In [12]:
%autoreload 2

In [13]:
collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
# Filter countries by geometry
countries = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
geometry = ee.Geometry.Point([24.06353794842853, -29.732969740562062])
sa = countries.filterBounds(geometry)
sa_geo = sa.geometry()

# Define start and end dates
startDate = ee.Date.fromYMD(2018, 7, 4)
endDate = ee.Date.fromYMD(2019, 8, 15)

In [7]:
X = preProcessXCollection(collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2'), region = sa_geo, startDate = startDate, endDate = endDate)
eeprint(X.limit(2))
eeprint(X.first().bandNames())

In [16]:
from data_extraction import extractFire

y = preProcessyCollection(collection = ee.ImageCollection("ESA/CCI/FireCCI/5_1"), region = sa_geo, startDate = startDate, endDate = endDate)
X = preProcessXCollection(collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED"), region = sa_geo, startDate = startDate, endDate = endDate)

eeprint(extractFire(y, sa_geo, 250))

fire_2018_08_01.csv: |          | [  0.0%] in 00:00 (eta:     ?)

fire_2018_09_01.csv: |          | [  0.0%] in 00:00 (eta:     ?)

fire_2018_10_01.csv: |          | [  0.0%] in 00:00 (eta:     ?)

fire_2018_11_01.csv: |          | [  0.0%] in 00:00 (eta:     ?)

fire_2018_12_01.csv: |          | [  0.0%] in 00:00 (eta:     ?)

...

fire_2019_02_01.csv: |          | [  0.0%] in 00:00 (eta:     ?)

fire_2019_03_01.csv: |          | [  0.0%] in 00:00 (eta:     ?)

fire_2019_04_01.csv: |          | [  0.0%] in 00:00 (eta:     ?)

Cancelling...


FileNotFoundError: [Errno 2] No such file or directory: 'fire_2019_04_01.csv'

In [ ]:
fire = ee.ImageCollection("ESA/CCI/FireCCI/5_1")
fire = fire.first()
from geeml.utils import eeprint
eeprint(fire.bandNames())